In [1]:
from dotenv import dotenv_values
import openai
from openai import OpenAI
import numpy as np
import os
import sqlite3

C:\Users\hp\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
api_key = dotenv_values("../api_data.env")
openai.api_key = api_key['OPEN_AI_KEY']

In [3]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", openai.api_key))

In [4]:
import sqlite3

conn = sqlite3.connect("alerting_db")
print("Opened database successfully")

Opened database successfully


In [5]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [6]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [7]:
database_schema_string

'Table: alerts_table\nColumns: alert_id, alert_date, trader_id, deal_number, model_name\nTable: deals_table\nColumns: alert_id, deal_date, deal_number, price, volume'

In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about Alerts and Deals in trading data. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

In [9]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

In [10]:
GPT_MODEL = "gpt-3.5-turbo"

In [16]:
assistant_system_prompt = """You are a chatbot. Your role is to answer user questions politely and competently.
You should follow these instructions to solve the case:
- Understand their question and get the relevant answer.
- Make use of the ongoing chat conversation to anwer question and do not just rely on tool.
- Answer only if you are fully confident of the answer, otherwise say that can you please frame the sentence again.
"""

In [22]:
#!/usr/bin/env python
import PySimpleGUI as sg

'''
A simple send/response chat window.  Add call to your send-routine and print the response
If async responses can come in, then will need to use a different design that uses PySimpleGUI async design pattern

Copyright 2023 PySimpleSoft, Inc. and/or its licensors. All rights reserved.

Redistribution, modification, or any other use of PySimpleGUI or any portion thereof is subject to the terms of the PySimpleGUI License Agreement available at https://eula.pysimplegui.com.

You may not redistribute, modify or otherwise use PySimpleGUI or its contents except pursuant to the PySimpleGUI License Agreement.

'''

sg.theme('GreenTan') # give our window a spiffy set of colors

layout = [[sg.Text('Your output will go here', size=(40, 1))],
          [sg.Output(size=(110, 20), font=('Helvetica 10'))],
          [sg.Multiline(size=(70, 5), enter_submits=True, key='-QUERY-', do_not_clear=False),
           sg.Button('SEND', button_color=(sg.YELLOWS[0], sg.BLUES[0]), bind_return_key=True),
           sg.Button('EXIT', button_color=(sg.YELLOWS[0], sg.GREENS[0]))]]

window = sg.Window('Chat window', layout, font=('Helvetica', ' 13'), default_button_element_size=(8,2), use_default_focus=False)

#conversation_messages = []
conversation_messages = [
    {
                "role": "system",
                "content": assistant_system_prompt
    }
]
try:
    while True:     # The Event Loop
        event, values = window.read()
        if event in (sg.WIN_CLOSED, 'EXIT'):            # quit if exit button or X
            break
        if event == 'SEND':
            user_question = values['-QUERY-'].rstrip()
            conversation_messages.append ({
            "role":"user", 
            "content": user_question
            })
            print(user_question)
            #conversation_messages.append(messages)

            response = client.chat.completions.create(
                model=GPT_MODEL, 
                messages=conversation_messages, 
                tools= tools, 
                tool_choice="auto"
            )

            # Append the message to messages list
            response_message = response.choices[0].message 


            # Step 2: determine if the response from the model includes a tool call.   
            tool_calls = response_message.tool_calls
            if tool_calls:
                # If true the model will return the name of the tool / function to call and the argument(s)  
                tool_call_id = tool_calls[0].id
                tool_function_name = tool_calls[0].function.name
                tool_query_string = eval(tool_calls[0].function.arguments)['query']

                print(tool_query_string)
                conversation_messages.append({
                    "role":"assistant", 
                    "content": tool_query_string
                    })

                # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    #             if tool_function_name == 'ask_database':
    #                 results = ask_database(conn, tool_query_string)

    #                 messages.append({
    #                     "role":"tool", 
    #                     "tool_call_id":tool_call_id, 
    #                     "name": tool_function_name, 
    #                     "content":results
    #                 })

    #                 # Step 4: Invoke the chat completions API with the function response appended to the messages list
    #                 # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
    #                 model_response_with_function_call = client.chat.completions.create(
    #                     model= GPT_MODEL,
    #                     messages=messages,
    #                 )  # get a new response from the model where it can see the function response
    #                 print(model_response_with_function_call.choices[0].message.content)
    #             else: 
    #                 print(f"Error: function {tool_function_name} does not exist")
            else:
                # Model did not identify a function to call, result can be returned to the user
                conversation_messages.append({
                    "role":"assistant", 
                    "content": response_message.content
                    })
                print(response_message.content)
except Exception as e:
    print("Exception occured :",e)
    
window.close()

In [ ]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": user_question
}]

response = client.chat.completions.create(
    model='gpt-3.5-turbo', 
    messages=messages, 
    tools= tools, 
    tool_choice="auto"
)

# Append the message to messages list
response_message = response.choices[0].message 
messages.append(response_message)

print(response_message)

In [ ]:
tool_calls = response_message.tool_calls
tool_calls

In [ ]:
# Step 2: determine if the response from the model includes a tool call.   
tool_calls = response_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = eval(tool_calls[0].function.arguments)['query']
    
    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)
        
        messages.append({
            "role":"tool", 
            "tool_call_id":tool_call_id, 
            "name": tool_function_name, 
            "content":results
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    print(response_message.content) 

In [ ]:
#pip install PySimpleGUI

In [ ]:
#!/usr/bin/env python
import PySimpleGUI as sg

'''
A simple send/response chat window.  Add call to your send-routine and print the response
If async responses can come in, then will need to use a different design that uses PySimpleGUI async design pattern

Copyright 2023 PySimpleSoft, Inc. and/or its licensors. All rights reserved.

Redistribution, modification, or any other use of PySimpleGUI or any portion thereof is subject to the terms of the PySimpleGUI License Agreement available at https://eula.pysimplegui.com.

You may not redistribute, modify or otherwise use PySimpleGUI or its contents except pursuant to the PySimpleGUI License Agreement.

'''

sg.theme('GreenTan') # give our window a spiffy set of colors

layout = [[sg.Text('Your output will go here', size=(40, 1))],
          [sg.Output(size=(110, 20), font=('Helvetica 10'))],
          [sg.Multiline(size=(70, 5), enter_submits=True, key='-QUERY-', do_not_clear=False),
           sg.Button('SEND', button_color=(sg.YELLOWS[0], sg.BLUES[0]), bind_return_key=True),
           sg.Button('EXIT', button_color=(sg.YELLOWS[0], sg.GREENS[0]))]]

window = sg.Window('Chat window', layout, font=('Helvetica', ' 13'), default_button_element_size=(8,2), use_default_focus=False)

while True:     # The Event Loop
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'EXIT'):            # quit if exit button or X
        break
    if event == 'SEND':
        query = values['-QUERY-'].rstrip()
        # EXECUTE YOUR COMMAND HERE
        print('The command you entered was {}'.format(query), flush=True)

window.close()

### Create a dummy database

In [ ]:
# conn = sqlite3.connect("alerting_db")
# c = conn.cursor()

# c.execute(
#     """
#     CREATE TABLE IF NOT EXISTS alerts_table
#     ([alert_id] TEXT PRIMARY KEY, [alert_date] TEXT, [trader_id] TEXT , [deal_number] TEXT , [model_name] TEXT )
#     """
# )

# c.execute(
#     """
#     CREATE TABLE IF NOT EXISTS deals_table
#     ([alert_id] INTEGER, [deal_date] TEXT, [deal_number] TEXT,[price] INTEGER,[volume] INTEGER)
#     """
# )

# conn.commit()

In [ ]:
# # Insert data into Alert table
# for row in alert_df.itertuples(index=False):
#     c.execute('''
#     INSERT INTO alerts_table (alert_id, alert_date, trader_id, deal_number, model_name)
#     VALUES (?, ?, ?, ?, ?)
#     ''', (row.alert_id, row.alert_date, row.trader_id, row.deal_number, row.model_name))

# # Insert data into Deal table
# for row in deal_df.itertuples(index=False):
#     c.execute('''
#     INSERT INTO deals_table (alert_id, deal_date, deal_number, price, volume)
#     VALUES (?, ?, ?, ?, ?)
#     ''', (row.alert_id, row.deal_date, row.deal_number, row.price, row.volume))

# # Commit the transaction
# conn.commit()

In [ ]:
# import sqlite3
# import pandas as pd

# conn = sqlite3.connect("alerting_db")
# c = conn.cursor()

# c.execute(
#     """
#     SELECT
#     *
#     FROM deals_table where deal_number = 'deal_0'
#     """
# )

# df = pd.DataFrame(c.fetchall(), columns=['alert_id', 'deal_date', 'deal_number', 'price', 'volume'])

# print(df)

In [ ]:
# import pandas as pd
# import numpy as np
# from datetime import datetime, timedelta
# import random
# import uuid

# # Generate random dates
# def random_date(start, end):
#     return start + timedelta(
#         seconds=random.randint(0, int((end - start).total_seconds())),
#     )

# # Parameters
# num_rows = 500
# start_date = datetime(2023, 1, 1)
# end_date = datetime(2023, 12, 31)

# # Generate data for Alert table
# alert_data = {
#     "alert_id": [str(uuid.uuid4()) for _ in range(num_rows)],
#     "alert_date": [random_date(start_date, end_date).strftime("%Y-%m-%d") for _ in range(num_rows)],
#     "trader_id": [f"trader_{i}" for i in range(num_rows)],
#     "deal_number": [f"deal_{i}" for i in range(num_rows)],
#     "model_name": [f"model_{random.randint(1, 5)}" for _ in range(num_rows)]
# }

# # Generate data for Deal table
# deal_data = {
#     "alert_id": alert_data["alert_id"],  # Same alert_id as in the Alert table
#     "deal_date": [random_date(start_date, end_date).strftime("%Y-%m-%d") for _ in range(num_rows)],
#     "deal_number": [f"deal_{i}" for i in range(num_rows)],
#     "price": [random.randint(100, 1000) for _ in range(num_rows)],
#     "volume": [random.randint(1, 100) for _ in range(num_rows)]
# }

# # Create DataFrames
# alert_df = pd.DataFrame(alert_data)
# deal_df = pd.DataFrame(deal_data)

# # Merge DataFrames on alert_id
# merged_df = pd.merge(alert_df, deal_df, on="alert_id")

# # Show the DataFrame


In [ ]:
# alert_df.to_csv('input_data/function_calling/alert_df.csv',index=False)
# deal_df.to_csv('input_data/function_calling/deal_df.csv',index=False)